In [1]:
from base.registry import RUNNER
import torch.nn as nn

@RUNNER.register_module()
class Runner(nn.Module):
    def __init__(self, a, b):
        self.a = a
        self.b = b

    def __call__(self, *args, **kwargs):
        print(self.a, self.b)
        return self.a + self.b
    
runner = RUNNER.build(dict(type='Runner', a=1, b=2))

In [1]:
from base.registry import MY_DATASETS
import os.path as osp
root = osp.join('data', 'sonar')
dataset = MY_DATASETS.build(dict(type="WaterTankDataset", root=root))

import torch
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=4, shuffle=True)
for output in dataloader:
    print(output.get('img').shape)
    print(output.get('mask').shape)
    break

torch.Size([4, 3, 480, 320])
torch.Size([4, 1, 480, 320])


In [1]:
import torch
import torch.nn as nn
import copy
from mmengine.optim import (OptimWrapper, OptimWrapperDict, _ParamScheduler,
                            build_optim_wrapper)
# from mmengine.registry import OPTIM_WRAPPERS
from tools.runner import Runner
import matplotlib.pyplot as plt

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(3, 3, 3)
        self.bn = nn.BatchNorm2d(3)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)

        return x

model = Model()

optimizer = dict(type='SGD', lr=0.01, momentum=0.9, weight_decay=0.0005)
optim_wrapper = dict(type='OptimWrapper', optimizer=optimizer, clip_grad=None)
param_scheduler = [
    dict(
        type='LinearLR', start_factor=1e-6, by_epoch=False, begin=0, end=1500),
    dict(
        type='PolyLR',
        eta_min=0.0,
        power=1.0,
        begin=1500,
        end=160000,
        by_epoch=False,
    )
]

optim = Runner._build_optimizer('self', model, copy.deepcopy(optim_wrapper))
scheduler = Runner._build_param_scheduler('self', optim, copy.deepcopy(param_scheduler))

""" 测试 """
# for i in range(3000):
#     # optim.step()
#     for _scheduler in scheduler:
#         _scheduler.step()
#     # print(optim.param_groups[0]['lr'])
#     plt.plot(i, optim.param_groups[0]['lr'], 'ro')

In [2]:
from base.registry import LOSSES
import utils.loos
import torch

# loss_decode=dict(
#             type='CrossEntropyLoss', use_sigmoid=False, loss_weight=1.0)
# loss_decode=dict(
#             type='DiceLoss', loss_weight=1.0)

loss_decode=[
        dict(type='CrossEntropyLoss', loss_name='loss_ce', loss_weight=1.0),
        dict(type='DiceLoss', loss_name='loss_dice', loss_weight=3.0)
    ]
loss = LOSSES.build(loss_decode)
loss__ = LOSSES.build(loss_decode)
print(type(loss__))

pred = torch.rand((1, 3, 4, 4), requires_grad=True)
target = torch.randint(0, 3, (1, 4, 4))

print(loss[0](pred, target))
print(loss[1](pred, target) + loss[0](pred, target))
print(optim)

<class 'mmengine.model.base_module.Sequential'>
tensor(1.0742, grad_fn=<MulBackward0>)
tensor(2.3245, grad_fn=<AddBackward0>)


NameError: name 'optim' is not defined

In [5]:
# 获取当前时间戳
import time
# 格式化为年月日
import datetime

def get_current_time():
    return datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%0d%H%M%S')

time

20240625171727


In [1]:
import torch
import torch.nn as nn
from base.registry import MODEL
import model as ModelZoo

cfg = dict(type='Model', a=1, b=2)
model = MODEL.build(cfg)

In [1]:
"""API Test"""

import argparse
import logging
import os
import os.path as osp

from mmengine.config import Config, DictAction
from mmengine.logging import print_log
from tools.runner import Runner



def main():
    # load config
    cfg = Config.fromfile('configs/api_test.py')
    print(cfg)
    print(cfg.train_dataset)
    print(cfg.param_scheduler)
    runner = Runner(cfg=cfg)
    # if args.cfg_options is not None:
    #     cfg.merge_from_dict(args.cfg_options)

if __name__ == '__main__':
    main()

Config (path: configs/api_test.py): {'dataset_type': 'WaterTankDataset', 'root': 'data/sonar', 'train_dataset': {'type': 'WaterTankDataset', 'root': 'data/sonar', 'split': 'train'}, 'test_dataset': {'type': 'WaterTankDataset', 'root': 'data/sonar', 'split': 'test'}, 'train_dataloader': {'batch_size': 4, 'num_workers': 4}, 'test_dataloader': {'batch_size': 1, 'num_workers': 1}, 'optimizer': {'type': 'SGD', 'lr': 0.01, 'momentum': 0.9, 'weight_decay': 0.0005}, 'optim_wrapper': {'type': 'OptimWrapper', 'optimizer': {'type': 'SGD', 'lr': 0.01, 'momentum': 0.9, 'weight_decay': 0.0005}, 'clip_grad': None}, 'param_scheduler': [{'type': 'LinearLR', 'start_factor': 1e-06, 'by_epoch': False, 'begin': 0, 'end': 1500}, {'type': 'PolyLR', 'eta_min': 0.0, 'power': 1.0, 'begin': 1500, 'end': 160000, 'by_epoch': False}], 'decode_loss': {'type': 'CrossEntropyLoss', 'use_sigmoid': False, 'loss_weight': 1.0}, 'model_cfg': {'type': 'Model', 'a': 1, 'b': 2}}
{'type': 'WaterTankDataset', 'root': 'data/sonar

e:\GitHubProject\semantic-segmentation-code\utils\loos\cross_entropy_loss.py:250: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


In [ ]:
""" 注意 """
# Initiate inner count of `optim_wrapper`.
self.optim_wrapper.initialize_count_status(
    self.model,
    self._train_loop.iter,  # type: ignore
    self._train_loop.max_iters)  # type: ignore

In [43]:
""" 注意: 多损失计算 """
loss = dict()
seg_logits = pred
seg_label = target 
loss_decode = loss__

for _decode in loss_decode:
    print(_decode)
    if _decode.loss_name not in loss:
        loss[_decode.loss_name] = _decode(
            seg_logits,
            seg_label)
    else:
        loss[_decode.loss_name] += _decode(
            seg_logits,
            seg_label)
        
print(loss)

CrossEntropyLoss(avg_non_ignore=False)
DiceLoss()
{'loss_ce': tensor(1.0806, grad_fn=<MulBackward0>), 'loss_dice': tensor(1.2545, grad_fn=<MulBackward0>)}


In [44]:
""" 更新参数代码 """
from mmengine.utils import is_list_of
from collections import OrderedDict

log_vars = []
for loss_name, loss_value in loss.items():
    if isinstance(loss_value, torch.Tensor):
        log_vars.append([loss_name, loss_value.mean()])
    elif is_list_of(loss_value, torch.Tensor):
        log_vars.append(
            [loss_name,
                sum(_loss.mean() for _loss in loss_value)])
    else:
        raise TypeError(
            f'{loss_name} is not a tensor or list of tensors')

loss = sum(value for key, value in log_vars if 'loss' in key)
log_vars.insert(0, ['loss', loss])
log_vars = OrderedDict(log_vars)  # type: ignore
optim.update_params(loss)
print(log_vars)

OrderedDict([('loss', tensor(2.3351, grad_fn=<AddBackward0>)), ('loss_ce', tensor(1.0806, grad_fn=<MeanBackward0>)), ('loss_dice', tensor(1.2545, grad_fn=<MeanBackward0>))])


In [7]:
import cv2
import os.path as osp

img = cv2.imread(osp.join('data\sonar\Images\marine-debris-aris3k-16.png'))
mask = cv2.imread(osp.join('data\sonar\Masks\marine-debris-aris3k-16.png'))

print(img.shape)
print(mask.shape)

(480, 320, 3)
(480, 320, 3)


In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models.resnet as resnet
# from base.registry import MODEL

class Model(nn.Module):
    def __init__(self, a, b):
        super().__init__()
        self.backbone = resnet.resnet18()
        self.decoder = nn.Conv2d(512, 12, 1)

    def forward(self, x):
        B, C, H, W = x.shape
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)
        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)
        x = self.decoder(x)
        x = F.interpolate(x, size=(H, W), mode='bilinear', align_corners=True)
        x = nn.Softmax(dim=1)(x)

        return x

x = torch.randn(4, 3, 480, 320).cuda()
model = Model(1,2).cuda() # Model(1, 2).cuda()
# model.fc = nn.Conv2d(256, 12, 1).cuda()
y = model(x)
print(y.shape)

torch.Size([4, 12, 480, 320])
